In [1]:
#import jax.numpy as np
#from jax import pmap
import numpy as np
from maxnorm.maxnorm_completion import *
from maxnorm.tenalg import *
import sparse
from itertools import product

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%load_ext autoreload

In [2]:
%autoreload

In [3]:
# create random, low-rank tensor
t = 3
n = 200
r = 4
#delta = 0.1
const = 10
sigma = 0.1
ndata =  const * r * t * n * np.log10(n)
U = [np.random.randn(n, r) for i in range(t)]
print("n data: %.2e" % ndata)
print("n data t/2: %.2e" % int(const * r * n**(t/2) * np.log10(n)))

n data: 5.52e+04
n data t/2: 2.60e+05


In [4]:
max_qnorm_ub(U)

69.35314165950732

In [5]:
observation_mask = sparse.random(tuple([n for i in range(t)]), ndata * n**(-t))
data = observation_mask.copy()

clean_data = kr_get_items(U, data.coords)

#clean_data = np.array(list(map(get_item_coords, [data.coords[:, i] for i in range(data.nnz)])))

# for i in range(data.nnz):
#     pred = get_item(U, data.coords[:, i])
#     data.data[i] = pred
# clean_data = data.data

clean_data_rms = np.sqrt(np.sum(clean_data)**2 / len(clean_data))

data.data = clean_data + np.random.randn(data.nnz) * sigma
print(data.nnz)

55224


In [6]:
print(10* 400**(3/2))
print(10 * r * t * 400)

80000.0
48000


In [7]:
data.coords

array([[  0,   0,   0, ..., 199, 199, 199],
       [  1,   1,   1, ..., 193, 194, 197],
       [ 42,  64, 129, ..., 133,  44, 113]])

In [8]:
clean_data_rmse = np.sqrt(loss(U, data) / data.nnz)
delta = 2 * clean_data_rmse
#np.sqrt(np.sum((data.data - clean_data)**2)) * 2 / np.sqrt(data.nnz) * 1.5

print("rms of data:        %f" % clean_data_rms)
print("delta parameter:    %f" % delta)
print("rmse of U true:     %f" % np.sqrt(loss(U, data) / data.nnz))

/home/kameron/local/anaconda3/lib/python3.7/site-packages/sparse/_coo/numba_extension.py:18: NumbaDeprecationWarning: No direct replacement for 'numba.targets' available. Visit https://gitter.im/numba/numba-dev to request help. Thanks!
  from numba.targets.imputils import impl_ret_borrowed, lower_constant
/home/kameron/local/anaconda3/lib/python3.7/site-packages/sparse/_coo/numba_extension.py:18: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.targets.imputils', please update to use 'numba.core.imputils' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.targets.imputils import impl_ret_borrowed, lower_constant
/home/kameron/local/anaconda3/lib/python3.7/site-packages/sparse/_coo/numba_extension.py:18: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'impl_ret_borrowed' requested from: 'numba.targets.imputils', please u

rms of data:        1.490529
delta parameter:    0.199941
rmse of U true:     0.099970


In [9]:
# from tensorly.contrib.sparse.decomposition import parafac

# mask = data != 0
# core, factors = parafac(data, r, mask=mask, init='random', verbose=True, tol=1e-3, n_iter_max=8)
# scale_mat = np.diag(core.todense()**(1/t))
# U = [factors[i].todense() @ scale_mat for i in range(t)]

In [26]:
%autoreload
Unew2, cost_arr = tensor_completion_maxnorm(data, r*3, delta * np.sqrt(data.nnz), U0=Unew2,
                                            init='svdrand', kappa=100, beta=0.1, tol=1e-4, max_iter=100)

Initial cost: 130.388463
Initial qnorm_ub: 105.505907
|| r || = 0.094976, delta = 46.985689
Entering inner loop for factor 0
qnorm_factr = 2.209738
initial objective 1.303885e+02
iteration 30, objective 1.298e+02, relative change 2.524e-04        
maximum iteration 30 reached with relative change in objective function 0.00025
Entering inner loop for factor 1
qnorm_factr = 37.403753
initial objective 1.298032e+02
iteration 30, objective 1.224e+02, relative change 4.786e-04        
maximum iteration 30 reached with relative change in objective function 0.00048
Entering inner loop for factor 2
qnorm_factr = 117.544955
initial objective 1.224438e+02
iteration 30, objective 1.173e+02, relative change 1.482e-04        
maximum iteration 30 reached with relative change in objective function 0.00015

Iteration 0 complete


scaled || r || = 0.108763, delta = 0.199941
Max-qnorm upper bound: 84.632659
Cost function:         1.173e+02


Entering inner loop for factor 0
qnorm_factr = 1.783478
initi

iteration 30, objective 1.066e+02, relative change 1.128e-04        
maximum iteration 30 reached with relative change in objective function 0.00011
Entering inner loop for factor 1
qnorm_factr = 31.670165
initial objective 1.066370e+02
iteration 30, objective 1.063e+02, relative change 2.771e-05        
maximum iteration 30 reached with relative change in objective function 2.8e-05
Entering inner loop for factor 2
qnorm_factr = 107.708108
initial objective 1.062866e+02
iteration 30, objective 1.059e+02, relative change 9.939e-06        
relative change in objective function 9.9e-06 is within tolerance 1e-05 after 30 iterations

Iteration 9 complete


scaled || r || = 0.108606, delta = 0.199941
Max-qnorm upper bound: 73.370231
Cost function:         1.059e+02


Entering inner loop for factor 0
qnorm_factr = 1.585581
initial objective 1.059071e+02
iteration 30, objective 1.057e+02, relative change 1.111e-04        
maximum iteration 30 reached with relative change in objective function 

iteration 30, objective 9.937e+01, relative change 9.706e-05        
maximum iteration 30 reached with relative change in objective function 9.7e-05
Entering inner loop for factor 1
qnorm_factr = 30.036348
initial objective 9.936670e+01
iteration 30, objective 9.912e+01, relative change 1.543e-05        
maximum iteration 30 reached with relative change in objective function 1.5e-05
Entering inner loop for factor 2
qnorm_factr = 101.660861
initial objective 9.911882e+01
iteration 10, objective 9.885e+01, relative change 9.693e-06        
relative change in objective function 9.7e-06 is within tolerance 1e-05 after 10 iterations

Iteration 18 complete


scaled || r || = 0.107490, delta = 0.199941
Max-qnorm upper bound: 66.978008
Cost function:         9.885e+01


Entering inner loop for factor 0
qnorm_factr = 1.474219
initial objective 9.884936e+01
iteration 30, objective 9.868e+01, relative change 9.572e-05        
maximum iteration 30 reached with relative change in objective function

iteration 30, objective 9.391e+01, relative change 8.214e-05        
maximum iteration 30 reached with relative change in objective function 8.2e-05
Entering inner loop for factor 1
qnorm_factr = 28.834001
initial objective 9.391299e+01
iteration 30, objective 9.372e+01, relative change 1.313e-05        
maximum iteration 30 reached with relative change in objective function 1.3e-05
Entering inner loop for factor 2
qnorm_factr = 96.812808
initial objective 9.372325e+01
iteration 7, objective 9.352e+01, relative change 8.939e-06        
relative change in objective function 8.9e-06 is within tolerance 1e-05 after 7 iterations

Iteration 27 complete


scaled || r || = 0.106418, delta = 0.199941
Max-qnorm upper bound: 62.284238
Cost function:         9.352e+01


Entering inner loop for factor 0
qnorm_factr = 1.392585
initial objective 9.352281e+01
iteration 30, objective 9.339e+01, relative change 8.055e-05        
maximum iteration 30 reached with relative change in objective function 8.

iteration 30, objective 8.984e+01, relative change 7.257e-05        
maximum iteration 30 reached with relative change in objective function 7.3e-05
Entering inner loop for factor 1
qnorm_factr = 27.966416
initial objective 8.984012e+01
iteration 12, objective 8.971e+01, relative change 9.846e-06        
relative change in objective function 9.8e-06 is within tolerance 1e-05 after 12 iterations
Entering inner loop for factor 2
qnorm_factr = 93.262857
initial objective 8.971397e+01
iteration 6, objective 8.958e+01, relative change 9.992e-06        
relative change in objective function 1e-05 is within tolerance 1e-05 after 6 iterations

Iteration 36 complete


scaled || r || = 0.105305, delta = 0.199941
Max-qnorm upper bound: 58.988298
Cost function:         8.958e+01


Entering inner loop for factor 0
qnorm_factr = 1.336205
initial objective 8.957730e+01
iteration 30, objective 8.946e+01, relative change 7.183e-05        
maximum iteration 30 reached with relative change in objective f

...

initial objective 8.453076e+01
iteration 1, objective 8.453e+01, relative change 9.719e-06        
relative change in objective function 9.7e-06 is within tolerance 1e-05 after 1 iterations
Entering inner loop for factor 1
qnorm_factr = 26.755924
initial objective 8.452994e+01
iteration 8, objective 8.448e+01, relative change 9.829e-06        
relative change in objective function 9.8e-06 is within tolerance 1e-05 after 8 iterations
Entering inner loop for factor 2
qnorm_factr = 90.127096
initial objective 8.447644e+01
iteration 5, objective 8.443e+01, relative change 8.273e-06        
relative change in objective function 8.3e-06 is within tolerance 1e-05 after 5 iterations

Iteration 63 complete


scaled || r || = 0.103161, delta = 0.199941
Max-qnorm upper bound: 55.069461
Cost function:         8.443e+01


Entering inner loop for factor 0
qnorm_factr = 1.258908
initial objective 8.442552e+01
iteration 1, objective 8.442e+01, relative change 9.790e-06        
relative change in objec

iteration 1, objective 8.358e+01, relative change 9.876e-06        
relative change in objective function 9.9e-06 is within tolerance 1e-05 after 1 iterations
Entering inner loop for factor 1
qnorm_factr = 26.491779
initial objective 8.358453e+01
iteration 6, objective 8.354e+01, relative change 9.286e-06        
relative change in objective function 9.3e-06 is within tolerance 1e-05 after 6 iterations
Entering inner loop for factor 2
qnorm_factr = 89.766921
initial objective 8.354267e+01
iteration 5, objective 8.350e+01, relative change 6.908e-06        
relative change in objective function 6.9e-06 is within tolerance 1e-05 after 5 iterations

Iteration 72 complete


scaled || r || = 0.102736, delta = 0.199941
Max-qnorm upper bound: 54.387451
Cost function:         8.350e+01


Entering inner loop for factor 0
qnorm_factr = 1.244920
initial objective 8.350197e+01
iteration 1, objective 8.350e+01, relative change 9.935e-06        
relative change in objective function 9.9e-06 is within

iteration 1, objective 8.280e+01, relative change 9.890e-06        
relative change in objective function 9.9e-06 is within tolerance 1e-05 after 1 iterations
Entering inner loop for factor 1
qnorm_factr = 26.266005
initial objective 8.279950e+01
iteration 5, objective 8.277e+01, relative change 9.806e-06        
relative change in objective function 9.8e-06 is within tolerance 1e-05 after 5 iterations
Entering inner loop for factor 2
qnorm_factr = 89.486884
initial objective 8.276533e+01
iteration 5, objective 8.273e+01, relative change 6.142e-06        
relative change in objective function 6.1e-06 is within tolerance 1e-05 after 5 iterations

Iteration 81 complete


scaled || r || = 0.102361, delta = 0.199941
Max-qnorm upper bound: 53.830213
Cost function:         8.273e+01


Entering inner loop for factor 0
qnorm_factr = 1.233748
initial objective 8.273241e+01
iteration 1, objective 8.273e+01, relative change 9.945e-06        
relative change in objective function 9.9e-06 is within

iteration 1, objective 8.213e+01, relative change 9.608e-06        
relative change in objective function 9.6e-06 is within tolerance 1e-05 after 1 iterations
Entering inner loop for factor 1
qnorm_factr = 26.067562
initial objective 8.213216e+01
iteration 5, objective 8.210e+01, relative change 8.844e-06        
relative change in objective function 8.8e-06 is within tolerance 1e-05 after 5 iterations
Entering inner loop for factor 2
qnorm_factr = 89.227842
initial objective 8.210348e+01
iteration 5, objective 8.208e+01, relative change 5.652e-06        
relative change in objective function 5.7e-06 is within tolerance 1e-05 after 5 iterations

Iteration 90 complete


scaled || r || = 0.102069, delta = 0.199941
Max-qnorm upper bound: 53.338246
Cost function:         8.208e+01


Entering inner loop for factor 0
qnorm_factr = 1.223955
initial objective 8.207586e+01
iteration 1, objective 8.208e+01, relative change 9.658e-06        
relative change in objective function 9.7e-06 is within

initial objective 8.166755e+01
iteration 1, objective 8.167e+01, relative change 9.968e-06        
relative change in objective function 1e-05 is within tolerance 1e-05 after 1 iterations
Entering inner loop for factor 1
qnorm_factr = 25.941676
initial objective 8.166674e+01
iteration 5, objective 8.164e+01, relative change 8.760e-06        
relative change in objective function 8.8e-06 is within tolerance 1e-05 after 5 iterations
Entering inner loop for factor 2
qnorm_factr = 89.159122
initial objective 8.164384e+01
iteration 5, objective 8.162e+01, relative change 8.258e-06        
relative change in objective function 8.3e-06 is within tolerance 1e-05 after 5 iterations

Iteration 99 complete


scaled || r || = 0.101767, delta = 0.199941
Max-qnorm upper bound: 53.053473
Cost function:         8.162e+01




In [ ]:
#print("loss of U parafac:  %f" % np.sqrt(loss(U,data) / data.nnz))
print("loss of U true:     %f" % np.sqrt(loss(U, data) / data.nnz))
print("loss of U max:      %f" % np.sqrt(loss(Unew2, data) / data.nnz))

In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(cost_arr)
plt.xlabel('iterate')
plt.ylabel('cost')

## Generalization error

In [ ]:
mse_gen2 = kr_dot(Unew2,Unew2) + kr_dot(U,U) - 2 * kr_dot(Unew2, U)
norm_true = kr_dot(U,U)
print("relative RMSE: %1.4e" % np.sqrt(mse_gen2/norm_true))

In [ ]:
fig, axs = plt.subplots(1,t, figsize=(6, 20))
for i in range(t):
    axs[i].imshow(Unew2[i])